In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:34:05.609907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:34:05.717535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:34:05.717550: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:34:05.738790: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:34:06.266205: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:34:10.475715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:34:10.476247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:34:10.476300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:34:10.476340: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:34:10.476378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 396ms/step - loss: 2.2568 - auc: 0.4857
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8173 - auc: 0.8197
Epoch 3/100
1/1 [==============================] - 0s 41ms/step - loss: 1.9065 - auc: 0.8423
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8729 - auc: 0.8573
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.7493 - auc: 0.8753
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5806 - auc: 0.8844
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4144 - auc: 0.8980
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3331 - auc: 0.8976
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2769 - auc: 0.9004
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2386 - auc: 0.8999
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1803 

1/1 [==============================] - 0s 24ms/step - loss: 0.2189 - auc: 0.9920
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2122 - auc: 0.9934
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2226 - auc: 0.9913
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2144 - auc: 0.9923
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2274 - auc: 0.9904
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2102 - auc: 0.9930
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2115 - auc: 0.9932
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2265 - auc: 0.9905
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2197 - auc: 0.9917
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2127 - auc: 0.9927
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 2.1087 - auc: 0.5004
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7414 - auc: 0.8114
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7941 - auc: 0.8441
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7869 - auc: 0.8535
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6532 - auc: 0.8662
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5060 - auc: 0.8758
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3959 - auc: 0.8711
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2958 - auc: 0.8761
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2352 - auc: 0.8812
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2383 - auc: 0.8632
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2446 - auc: 0.9895
Epoch 89/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2518 - auc: 0.9879
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2432 - auc: 0.9897
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2452 - auc: 0.9896
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2379 - auc: 0.9900
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2453 - auc: 0.9890
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2468 - auc: 0.9884
Epoch 95/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2502 - auc: 0.9883
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2481 - auc: 0.9878
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2460 - auc: 0.9889
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 2.1720 - auc: 0.4612
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7182 - auc: 0.8013
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7966 - auc: 0.8387
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7983 - auc: 0.8574
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6799 - auc: 0.8691
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5297 - auc: 0.8841
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 1.3816 - auc: 0.8924
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2460 - auc: 0.9060
Epoch 9/100
1/1 [==============================] - 0s 39ms/step - loss: 1.1751 - auc: 0.9110
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1690 - auc: 0.8979
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2291 - auc: 0.9906
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2282 - auc: 0.9902
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2225 - auc: 0.9911
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2197 - auc: 0.9922
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2206 - auc: 0.9921
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2179 - auc: 0.9924
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2221 - auc: 0.9906
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2254 - auc: 0.9908
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2245 - auc: 0.9906
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2112 - auc: 0.9931
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.7659 - auc: 0.7788
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5964 - auc: 0.7877
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4653 - auc: 0.8700
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3977 - auc: 0.8838
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2553 - auc: 0.9112
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.1484 - auc: 0.9333
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0972 - auc: 0.9384
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0047 - auc: 0.9602
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 0.9435 - auc: 0.9691
Epoch 10/100
1/1 [==============================] - 0s 38ms/step - loss: 0.8897 - auc: 0.9756
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2059 - auc: 0.9929
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2112 - auc: 0.9924
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1957 - auc: 0.9945
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2095 - auc: 0.9918
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2096 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2023 - auc: 0.9930
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2152 - auc: 0.9907
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2021 - auc: 0.9926
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2113 - auc: 0.9916
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2118 - auc: 0.9918
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.8051 - auc: 0.7495
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5985 - auc: 0.8052
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4697 - auc: 0.8574
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3296 - auc: 0.8895
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2511 - auc: 0.9073
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1300 - auc: 0.9420
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0593 - auc: 0.9550
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 0.9877 - auc: 0.9658
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9315 - auc: 0.9723
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8778 - auc: 0.9770
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2014 - auc: 0.9933
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2014 - auc: 0.9917
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1986 - auc: 0.9930
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1977 - auc: 0.9926
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1974 - auc: 0.9930
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1924 - auc: 0.9933
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1917 - auc: 0.9939
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1877 - auc: 0.9943
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1975 - auc: 0.9928
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1953 - auc: 0.9928
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 3.6107 - auc: 0.2309
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.0367 - auc: 0.5368
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6028 - auc: 0.8364
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7329 - auc: 0.8648
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8459 - auc: 0.8754
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9780 - auc: 0.8681
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9904 - auc: 0.8664
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9240 - auc: 0.8732
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8389 - auc: 0.8819
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6634 - auc: 0.8992
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2820 - auc: 0.9864
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2754 - auc: 0.9876
Epoch 90/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2747 - auc: 0.9873
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2796 - auc: 0.9862
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2809 - auc: 0.9852
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2774 - auc: 0.9852
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2804 - auc: 0.9848
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2844 - auc: 0.9844
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2710 - auc: 0.9869
Epoch 97/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2732 - auc: 0.9866
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 349ms/step - loss: 2.4190 - auc: 0.3528
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6742 - auc: 0.7857
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7413 - auc: 0.8410
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8087 - auc: 0.8532
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7988 - auc: 0.8655
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 1.6595 - auc: 0.8828
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5378 - auc: 0.8883
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3847 - auc: 0.8954
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2683 - auc: 0.9011
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1710 - auc: 0.9107
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2350 - auc: 0.9907
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2352 - auc: 0.9913
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2320 - auc: 0.9912
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2349 - auc: 0.9913
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2292 - auc: 0.9915
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2387 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2410 - auc: 0.9894
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2353 - auc: 0.9906
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2445 - auc: 0.9882
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2345 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 2.0263 - auc: 0.5382
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6446 - auc: 0.8311
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7232 - auc: 0.8557
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6996 - auc: 0.8705
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5808 - auc: 0.8802
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 1.4096 - auc: 0.8941
Epoch 7/100
1/1 [==============================] - 0s 49ms/step - loss: 1.2805 - auc: 0.9064
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2077 - auc: 0.9065
Epoch 9/100
1/1 [==============================] - 0s 46ms/step - loss: 1.2092 - auc: 0.8835
Epoch 10/100
1/1 [==============================] - 0s 47ms/step - loss: 1.1684 - auc: 0.8893
Epoch 11/100
1/1 [==============================] - 0s 40ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2422 - auc: 0.9894
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2474 - auc: 0.9886
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2363 - auc: 0.9901
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2420 - auc: 0.9894
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2439 - auc: 0.9891
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2387 - auc: 0.9895
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2402 - auc: 0.9902
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2276 - auc: 0.9919
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2367 - auc: 0.9897
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2350 - auc: 0.9902
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 618ms/step - loss: 3.0447 - auc: 0.2529
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8383 - auc: 0.6392
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5795 - auc: 0.8552
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7219 - auc: 0.8704
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8239 - auc: 0.8789
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8281 - auc: 0.8852
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8110 - auc: 0.8858
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7082 - auc: 0.8947
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5892 - auc: 0.9045
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4215 - auc: 0.9170
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2354 - auc: 0.9925
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2327 - auc: 0.9922
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2451 - auc: 0.9890
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2382 - auc: 0.9907
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2289 - auc: 0.9918
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2338 - auc: 0.9915
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2249 - auc: 0.9927
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2230 - auc: 0.9933
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2290 - auc: 0.9913
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2263 - auc: 0.9919
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [12]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 345ms/step - loss: 1.9143 - auc: 0.7769
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7338 - auc: 0.7371
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5778 - auc: 0.7702
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4553 - auc: 0.8355
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 1.4038 - auc: 0.8630
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2735 - auc: 0.8980
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1906 - auc: 0.9113
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0881 - auc: 0.9330
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0435 - auc: 0.9374
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9687 - auc: 0.9540
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2228 - auc: 0.9917
Epoch 89/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2233 - auc: 0.9919
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2258 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2203 - auc: 0.9924
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2323 - auc: 0.9892
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2272 - auc: 0.9905
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2206 - auc: 0.9917
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2146 - auc: 0.9931
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2286 - auc: 0.9911
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2215 - auc: 0.9908
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [13]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('9feature_remove_one2.csv',index=False)
print(de)

                 model       auc
0            all_model  0.946730
1    remove_AtomPairFP  0.928967
2  remove_Connectivity  0.946906
3        remove_Charge  0.947629
4         remove_Kappa  0.949202
5          remove_MAP4  0.943673
6           remove_MOE  0.944610
7      remove_MorganFP  0.946253
8      remove_Topology  0.946250
9     remove_TorsionFP  0.947784
